In [1]:
import re
import pandas as pd
from os.path import join
from pandas import DataFrame
from pandas import HDFStore

### 0. 准备工作

#### (1). 通用字段定义

In [2]:
typeDefines = {u'dev_id': str, u'stat_date': str, u'stat_time': str, u'post_id': int, 'type': int, 'stat_view': int, 'stat_click': int}
header = ['dev_id', 'stat_date', 'stat_time', 'post_id', 'type', 'stat_view', 'stat_click']
dtypes = [typeDefines[col] for col in header]
dbName = 'db.h5'

####  (2). 获得数据文件列表

In [3]:
fileList = !ls data
fileList = filter(lambda x: re.match(r'^\d{6}_\d$', x), fileList)
fileList = map(lambda x: join('data', x), fileList)

In [4]:
def remove(dfName):
    s = HDFStore(dbName)
    if s.get_node(dfName):
        s.remove(dfName)
    s.close()

### 2. 生成train数据集

In [5]:
dfName = 'page_view_train'

### (1) 读取train清单

In [6]:
train = pd.read_csv('data/train.txt', sep='\t', names=[u'dev_id', u'post_id', u'praise', u'time'])
train = train.sort_values([u'dev_id', u'post_id', u'time', u'praise'])
train = train.groupby([u'dev_id', u'post_id'])['praise'].last().reset_index()
trainList = train[[u'dev_id', u'post_id']].drop_duplicates()

### (2). 定义处理函数

In [7]:
def load_page_view_train(filename):
    """ """
    print '正在处理: %s' % filename, 
    # 按每次100万条方式处理, 数据文件
    for df in pd.read_csv(filename ,sep='\t', names=header, chunksize=1000000, converters=typeDefines):
        #  关联上train过滤调多余数据
        df = trainList.merge(df)
        # 导出到hdf文件
        df.to_hdf(dbName, dfName, append=True)
        print '.',
    print '(ok)'

### 3. 生成train数据清单

#### (1). 清空数据集

In [8]:
remove(dfName)

#### (2) 读取数据

In [9]:
for filename in fileList[-1:]:
    load_page_view_train(filename)

正在处理: data/000007_0 . . . (ok)


### 4. 读取整理过的数据集

In [10]:
page_view_train = pd.read_hdf(dbName, dfName)
len(page_view_train)

27141

### 5. 统计

In [11]:
g = page_view_train.groupby(['dev_id', 'post_id'])
d = g[['stat_view', 'stat_click']].sum()
d['days'] = g['stat_date'].nunique()

In [12]:
c1 = page_view_train['type'] == 1
g1 = page_view_train[c1].groupby(['dev_id', 'post_id'])
d1 = g1[['stat_view', 'stat_click']].sum()
d1['days'] = g1['stat_date'].nunique()

In [13]:
c2 = page_view_train['type'] == 2
g2 = page_view_train[c2].groupby(['dev_id', 'post_id'])
d2 = g2[['stat_view', 'stat_click']].sum()
d2['days'] = g2['stat_date'].nunique()

In [14]:
d = d.join(d1, rsuffix=('_1'))
d = d.join(d2, rsuffix=('_2'))
d = d.fillna(0)

In [15]:
d = d.reset_index()
d1 = d1.reset_index()
d2 = d2.reset_index()

In [16]:
len(d[['dev_id','post_id']].drop_duplicates())

20239

### 6. 保存到训练集中

In [17]:
dfName = 'train'
remove(dfName)
train.merge(d).to_hdf(dbName, dfName, append=True)